In [2]:
import json
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import mplhep as mh

from hist.hist import Hist
from hist.axis import Regular
from hist.axis import IntCategory
from hist.axis import StrCategory

from pathlib import Path

from typing import Optional, Union

from matplotlib.colors import Colormap, ListedColormap
from matplotlib.colors import LogNorm

In [11]:
def get_zero_eff_det(
    data,
    geom,
    roll_blacklist,
):
    roll_names = np.unique(geom['roll_name'])
    is_blacklist = np.vectorize(lambda item: item in roll_blacklist)
    blacklist_mask = ~is_blacklist(roll_names)
    roll_names = roll_names[blacklist_mask]
    h_total = data['total'].to_hist()
    h_passed = data['passed'].to_hist()
    total = h_total[roll_names.tolist()].values()
    passed = h_passed[roll_names.tolist()].values()
    eff = np.divide(passed, total, out=np.zeros_like(total),
                    where=(total > 0)) * 100
    hist, bins = np.histogram(eff, bins=101, range=(0, 101))

    eff_over70 = eff[eff > 70].mean()
    n_rolls = len(roll_names)
    n_rolls_over70 = len(roll_names[eff > 70])
    n_rolls_over0_under70 = len(roll_names[(eff <= 70) & (eff > 0)])
    n_rolls_0 = len(roll_names[eff == 0])

    zero_eff_rolls = roll_names[eff == 0]
    return zero_eff_rolls

In [18]:
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')
data_2022_path = working_dir / 'data' / 'Run2022.root'
data_2023_path = working_dir / 'data' / 'Run2023.root' 
geom_path = working_dir / 'geometry' / 'run3.csv'
roll_blacklist_path = working_dir / 'blacklist' / 'roll-blacklist.json'

if roll_blacklist_path is None:
    roll_blacklist = set()
else:
    with open(roll_blacklist_path) as stream:
        roll_blacklist = set(json.load(stream))


data_2022 = uproot.open(data_2022_path)
data_2023 = uproot.open(data_2023_path)

geom = pd.read_csv(geom_path)

zero_eff_rolls_2022 = get_zero_eff_det(data_2022, geom, roll_blacklist)
zero_eff_rolls_2023 = get_zero_eff_det(data_2023, geom, roll_blacklist)

df_2022 = pd.DataFrame(zero_eff_rolls_2022)
df_2023 = pd.DataFrame(zero_eff_rolls_2023)

df_2022.to_csv(working_dir / 'plotting' / 'zero_eff_det_2022.csv', index=False)
df_2023.to_csv(working_dir / 'plotting' / 'zero_eff_det_2023.csv', index=False)



/users/eigen1907/micromamba/envs/ds4hep/lib/python3.11/site-packages/hist/basehist.py:400: UserWarning: List indexing selection is experimental. Removed bins are not placed in overflow.
  return super().__getitem__(self._index_transform(index))


In [6]:
data = uproot.open("/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting/data/Run3.root:tree").arrays('residual_x', library='np')

In [2]:
import pandas as pd

In [9]:
zero2022 = pd.read_csv("/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting/plotting/zero_eff_det_2022.csv", index_col=False)
zero2023 = pd.read_csv("/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting/plotting/zero_eff_det_2023.csv", index_col=False)

zero2022 = set(zero2022['0'])
zero2023 = set(zero2023['0'])

print(zero2022)
print(zero2023)

{'W+2_RB3+_S12_Forward', 'W+0_RB4-_S07_Forward', 'W-1_RB1in_S10_Forward', 'W-2_RB1out_S09_Backward', 'W+2_RB4+_S08_Forward', 'W+1_RB2out_S03_Forward', 'W-2_RB2out_S01_Forward', 'RE+1_R3_CH15_B', 'W+0_RB3+_S11_Forward', 'W+0_RB2out_S08_Backward', 'W+0_RB3+_S04_Forward', 'W+0_RB3+_S04_Backward', 'W-1_RB1out_S10_Backward', 'W-2_RB2in_S04_Backward', 'W-1_RB3-_S02_Backward', 'RE+1_R2_CH35_A', 'W-2_RB3-_S10_Backward', 'W-2_RB4+_S04_Forward', 'W-1_RB4-_S04_Backward', 'W-2_RB4-_S07_Backward', 'W-1_RB3-_S10_Forward', 'RE+1_R3_CH04_B', 'W+0_RB3+_S11_Backward', 'W-2_RB2out_S01_Middle', 'W+0_RB2in_S07_Backward', 'W-1_RB2out_S07_Backward', 'W-2_RB3+_S10_Forward', 'W+2_RB2in_S08_Backward', 'W+1_RB4--_S04_Backward', 'W-2_RB2in_S04_Forward', 'W-1_RB1in_S03_Forward', 'RE+1_R3_CH15_A', 'W+2_RB3-_S09_Backward', 'W-1_RB1in_S02_Forward', 'RE+1_R3_CH04_C', 'W+0_RB2out_S04_Backward', 'W-2_RB3-_S11_Forward', 'W-2_RB4-_S04_Backward', 'W+0_RB1in_S10_Forward', 'W+1_RB1out_S05_Forward', 'W+2_RB3-_S12_Backward', '

In [21]:
from pathlib import Path
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')
print(zero2022 & zero2023)
df_both = pd.DataFrame({"roll_name": sorted(list(zero2022 & zero2023))})
df_both.to_csv(working_dir / 'plotting' / 'both_zero_rolls.csv', index=False)

{'W+2_RB3+_S12_Forward', 'W+0_RB4-_S07_Forward', 'W-1_RB1in_S10_Forward', 'W-2_RB1out_S09_Backward', 'W+2_RB4+_S08_Forward', 'W+1_RB2out_S03_Forward', 'W-2_RB2out_S01_Forward', 'RE+1_R3_CH15_B', 'W+0_RB3+_S11_Forward', 'W+0_RB2out_S08_Backward', 'W+0_RB3+_S04_Forward', 'W+0_RB3+_S04_Backward', 'W-1_RB1out_S10_Backward', 'W-2_RB2in_S04_Backward', 'W-1_RB3-_S02_Backward', 'RE+1_R2_CH35_A', 'W-2_RB3-_S10_Backward', 'W-2_RB4+_S04_Forward', 'W-1_RB4-_S04_Backward', 'W-2_RB4-_S07_Backward', 'W-1_RB3-_S10_Forward', 'RE+1_R3_CH04_B', 'W+0_RB3+_S11_Backward', 'W-2_RB2out_S01_Middle', 'W+0_RB2in_S07_Backward', 'W-1_RB2out_S07_Backward', 'W-2_RB3+_S10_Forward', 'W+2_RB2in_S08_Backward', 'W+1_RB4--_S04_Backward', 'W-2_RB2in_S04_Forward', 'W-1_RB1in_S03_Forward', 'RE+1_R3_CH15_A', 'W+2_RB3-_S09_Backward', 'W-1_RB1in_S02_Forward', 'RE+1_R3_CH04_C', 'W+0_RB2out_S04_Backward', 'W-2_RB3-_S11_Forward', 'W-2_RB4-_S04_Backward', 'W+0_RB1in_S10_Forward', 'W+1_RB1out_S05_Forward', 'W+2_RB3-_S12_Backward', '

In [20]:
print(zero2022.difference(zero2023))
df_only2022 = pd.DataFrame({"roll_name": sorted(list(zero2022.difference(zero2023)))})
df_only2022.to_csv(working_dir / 'plotting' / 'only2022_zero_rolls.csv', index=False)

{'W-2_RB3-_S06_Forward', 'W-2_RB2in_S01_Backward', 'W-2_RB4++_S04_Forward', 'W-2_RB2in_S01_Forward', 'W+1_RB3+_S08_Backward', 'W-2_RB4++_S04_Backward', 'RE+2_R3_CH33_B', 'RE+2_R3_CH33_C', 'W-2_RB3-_S06_Backward', 'W-2_RB4+_S07_Backward', 'W+0_RB2in_S06_Middle', 'W-2_RB4+_S07_Forward', 'W+2_RB1out_S07_Backward', 'W+2_RB3-_S06_Backward', 'W+2_RB1out_S07_Forward', 'W+1_RB3+_S08_Forward'}


In [22]:
print(zero2023.difference(zero2022))
df_only2023 = pd.DataFrame({"roll_name": sorted(list(zero2023.difference(zero2022)))})
df_only2023.to_csv(working_dir / 'plotting' / 'only2023_zero_rolls.csv', index=False)

{'W+0_RB1in_S05_Forward', 'W-2_RB2in_S03_Backward', 'W-2_RB4-_S05_Forward', 'W-1_RB2out_S05_Forward', 'W+1_RB2out_S02_Forward', 'W-2_RB3-_S12_Backward', 'W-1_RB4++_S04_Forward', 'W-2_RB2out_S03_Forward', 'W-2_RB2out_S03_Backward', 'RE+1_R3_CH04_A', 'W+0_RB3-_S09_Forward', 'W+1_RB2in_S02_Middle', 'W-2_RB1out_S01_Forward', 'W+1_RB2in_S02_Forward', 'W+1_RB2out_S05_Backward', 'W-1_RB2in_S10_Forward', 'W+2_RB3+_S07_Forward', 'W-1_RB4+_S04_Backward', 'W-2_RB3+_S11_Backward', 'W-2_RB4-_S12_Backward', 'W-1_RB4+_S07_Forward', 'W-2_RB2out_S11_Middle', 'W+0_RB4++_S04_Forward', 'W-2_RB2in_S03_Forward', 'W-2_RB2out_S11_Backward', 'W-1_RB2out_S04_Backward', 'W-2_RB2in_S05_Forward', 'W-1_RB2out_S10_Forward', 'W+0_RB4+_S04_Backward', 'W-2_RB2out_S03_Middle', 'W-1_RB2in_S10_Middle', 'W-1_RB2in_S10_Backward', 'W+1_RB2out_S05_Forward', 'W-2_RB2out_S11_Forward', 'W-2_RB4-_S05_Backward', 'W+0_RB3+_S09_Backward', 'W-1_RB4++_S04_Backward', 'W-2_RB1out_S08_Forward', 'W-2_RB1in_S12_Backward', 'W-1_RB2out_S05_B